![clothing_classification](clothing_classification.png)


Fashion Forward is a new AI-based e-commerce clothing retailer.
They want to use image classification to automatically categorize new product listings, making it easier for customers to find what they're looking for. It will also assist in inventory management by quickly sorting items.

As a data scientist tasked with implementing a garment classifier, your primary objective is to develop a machine learning model capable of accurately categorizing images of clothing items into distinct garment types such as shirts, trousers, shoes, etc.

In [1]:
# Run the cells below first

In [2]:
        !pip install torchmetrics
        !pip install torchvision

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy, Precision, Recall

In [4]:
# Load datasets
from torchvision import datasets
import torchvision.transforms as transforms

transform = transforms.Compose( [transforms.ToTensor(), transforms.Resize((28, 28))])
train_data = datasets.FashionMNIST(root='./data',    train=True, download=True, transform=transform)
test_data = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transforms.ToTensor())

In [5]:
transform = transforms.Compose([
    transforms.ToTensor()])
train_data = datasets.FashionMNIST(root='D:/sci_fi/Programming/AUA/classes/ai/deep_learning/practice/data', train = True, download = True, transform = transform)
test_data = datasets.FashionMNIST(root = 'D:/sci_fi/Programming/AUA/classes/ai/deep_learning/practice/data', train = False, download = True, transform=transforms.ToTensor())

In [6]:
train_dataloader = DataLoader(train_data, shuffle = True, batch_size =128)
test_dataloader = DataLoader(test_data, shuffle = False, batch_size = 128)
img, lab = next(iter(train_dataloader))
print(lab)
print(len(lab)== 128)

tensor([7, 8, 7, 3, 0, 2, 9, 0, 6, 5, 0, 4, 3, 7, 9, 7, 0, 1, 2, 5, 8, 9, 5, 7,
        0, 9, 9, 8, 3, 3, 0, 2, 1, 5, 3, 2, 5, 8, 9, 5, 6, 5, 4, 4, 5, 8, 3, 1,
        2, 1, 9, 1, 1, 5, 5, 6, 7, 2, 3, 2, 0, 6, 6, 2, 4, 8, 9, 8, 4, 0, 9, 0,
        5, 1, 2, 6, 2, 2, 7, 0, 5, 4, 2, 0, 5, 0, 7, 0, 3, 2, 1, 9, 9, 5, 9, 8,
        8, 7, 7, 4, 9, 3, 4, 1, 7, 6, 8, 1, 9, 0, 5, 3, 7, 8, 1, 0, 4, 3, 2, 0,
        7, 8, 3, 4, 2, 9, 0, 0])
True


In [7]:
print(type(train_data))
print(type(test_data))
print(len(train_data))
print(len(test_data))
print(img.size)
un_lab = np.unique([lab for _,lab in train_data])
print(un_lab)


<class 'torchvision.datasets.mnist.FashionMNIST'>
<class 'torchvision.datasets.mnist.FashionMNIST'>
60000
10000
<built-in method size of Tensor object at 0x7ff4880fbb30>
[0 1 2 3 4 5 6 7 8 9]


In [8]:
class ConvFashion(nn.Module):
  def __init__(self, num_class):
    super().__init__()
    self.feature_extractor = nn.Sequential(
        nn.Conv2d(1, 32, kernel_size =3),
        nn.ELU(),
        nn.MaxPool2d(kernel_size = 3, padding=1),
        nn.Conv2d(32, 64, kernel_size = 3),
        nn.ELU(),
        nn.MaxPool2d(kernel_size = 3, padding = 1),
        nn.Flatten(),
    )
    dummy_input = torch.randn(1, 1, 28, 28)  
    output_size = self.feature_extractor(dummy_input).shape[1]
    self.classifier = nn.Linear(output_size, num_class)

  def forward(self, x):
    x = self.feature_extractor(x)
    pred = self.classifier(x)
    return pred

In [9]:
num_classes = len(un_lab)
print(num_classes)
net = ConvFashion(num_classes)
epochs = 2
print(net.parameters())
optimizer = optim.SGD(net.parameters(), lr = 0.001)
criterion  = nn.CrossEntropyLoss()
for epoch in range(epochs):
  running_loss = 0.0
  for img, lab in train_dataloader:
    optimizer.zero_grad()
    pred = net(img)
    loss = criterion(pred, lab)
    loss.backward()
    optimizer.step()
    running_loss +=loss.item()
  print(f'Epoch [{epoch + 1}/{epochs}], Loss: {running_loss / len(train_dataloader):.4f}')

10
<generator object Module.parameters at 0x7ff48aaf8b30>
Epoch [1/2], Loss: 2.2321
Epoch [2/2], Loss: 2.0805


In [10]:
recall = Recall(task = "multiclass", num_classes = num_classes, average = "macro")
precision = Precision(num_classes = num_classes, task = "multiclass", average = "macro")
accuracy = Accuracy(num_classes = num_classes, task = "multiclass", average = "macro")
predictions = []
net.eval()
with torch.no_grad():
  for img, lab in test_dataloader:
    predictions = net(img)
    _, pred = torch.max(predictions, 1)
    recall(pred, lab)
    precision(pred, lab)
    accuracy(pred, lab)
recall = recall.compute()
precision = precision.compute()
accuracy = accuracy.compute()
predictions = predictions.tolist()
print(f'Recall: {recall:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Accuracy: {accuracy:.4f}')
print(type(predictions))


Recall: 0.4889
Precision: 0.5752
Accuracy: 0.4889
<class 'list'>
